In [154]:
import pandas as pd
from bs4 import BeautifulSoup
import sqlalchemy, sqlite3

In [155]:
engine = sqlalchemy.create_engine('sqlite:///data_db.db')
file_to_read = "../media/20210601-1812_prototype1_wifi_scan-01.csv"

In [156]:
with open(file_to_read, "r") as f:
    df = pd.read_csv(f)

In [157]:
print(df)

               BSSID       First time seen        Last time seen  channel  \
0  A6:08:F5:68:D5:17   2021-06-01 18:12:35   2021-06-01 18:12:39        1   
1  A4:08:F5:68:D3:16   2021-06-01 18:12:35   2021-06-01 18:12:39        1   
2  7A:97:41:0F:33:D7   2021-06-01 18:12:38   2021-06-01 18:12:39        6   
3  70:97:41:0F:33:D3   2021-06-01 18:12:38   2021-06-01 18:12:39        6   
4  7A:97:41:0F:33:D4   2021-06-01 18:12:38   2021-06-01 18:12:39        6   
5        Station MAC       First time seen        Last time seen    Power   

        Speed  Privacy          Cipher  Authentication   Power   # beacons  \
0          54      OPN                                   -57.0         5.0   
1          54     WPA2            CCMP             PSK   -57.0         5.0   
2          54     WPA2            CCMP             PSK   -26.0         3.0   
3          54     WPA2            CCMP             PSK   -27.0         3.0   
4          54      OPN                                   -27.0        

In [158]:
mac_list = [mac for mac in df.iloc[:-1,0]] # rows[first to last], columns[first, none other]
print(mac_list)


['A6:08:F5:68:D5:17', 'A4:08:F5:68:D3:16', '7A:97:41:0F:33:D7', '70:97:41:0F:33:D3', '7A:97:41:0F:33:D4']


In [159]:
# df.to_sql('test', engine, if_exists='append', index=False)
# was causing errors - save it later so we don't get duplicates that fuck things up

In [160]:
# macs = pd.read_sql('test', engine) # - see above
macs = df['BSSID']
macs = [mac for mac in macs if mac != "Station MAC"] # csv/df needs cleaning, unpredictable second header row
print(macs)

['A6:08:F5:68:D5:17', 'A4:08:F5:68:D3:16', '7A:97:41:0F:33:D7', '70:97:41:0F:33:D3', '7A:97:41:0F:33:D4']


In [161]:
mac_prefixes = [mac[:8].replace(":", "") for mac in macs] # organisational identifier
mac_suffixes = [mac[9:].replace(":", "") for mac in macs] # personal identifier
print("prefixes")
print(mac_prefixes)
print("suffixes")
print(mac_suffixes)

prefixes
['A608F5', 'A408F5', '7A9741', '709741', '7A9741']
suffixes
['68D517', '68D316', '0F33D7', '0F33D3', '0F33D4']


In [162]:
import requests

In [163]:
def mac_lookup(macaddr):
    headers = {'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}
    r = requests.get(f'https://maclookup.app/search/result?mac={macaddr}', headers = headers)
    c = r.content
    soup = BeautifulSoup(c, "html.parser")
    try:
        org = soup.select_one("h2").get_text()
    except:
        org="Unknown"
    return org

In [164]:
mac_lookup("A408F5")

'Sagemcom Broadband SAS'

In [165]:
devices = []
devices = [mac_lookup(prefix) for prefix in mac_prefixes]
print(devices)

['Unknown', 'Sagemcom Broadband SAS', 'Unknown', 'Arcadyan Corporation', 'Unknown']


### Clean and store CSV data to DB

In [166]:
data = pd.read_csv(file_to_read, sep=r'\s*,\s*', encoding='ascii', engine='python')

# delete second header row
data.drop(data.index[data['BSSID'] == "Station MAC"], inplace=True)

# put first seen and last seen into a list of 'sightings' of that device
sightings_dict = {}
for device, fts, lts in zip(data['BSSID'], data['First time seen'], data['Last time seen']):
    sightings = []
    sightings.append(fts)
    sightings.append(lts)
    sightings_dict[device] = sightings
    
# we can do this with power as well so we can see whether device is moving closer/further from detector
power_dict = {}
for device, power_reading in zip(data['BSSID'], data['Power']):
    power_readings = []
    if not pd.isna(power_reading):
        power_readings.append(power_reading)
    power_dict[device] = power_readings

# sort out devices that appeared below second heaader row (non-dynamically...)
for index, row in data.iterrows():
    if int(row.channel) < 0:
        power = float(data.at[index, 'channel'])
        print(f"Channel reading is {power}")
        # data.at[index, 'Power'] = power
        data.at[index, 'channel'] = 99
        if len(power_dict[row.BSSID]) == 0:
            power_dict[row.BSSID] = power
            print(f"NEW dict is  {power_dict[row.BSSID]}")
        else:
            power_dict[row.BSSID].append(power)
            print(f"Power dict is now {power_dict[row.BSSID]}")

    

data = data.assign(sightings=[str(value) for value in sightings_dict.values()])
data = data.assign(power_readings=[str(value) for value in power_dict.values()])
data = data.assign(maker=[m for m in devices])
data = data[['BSSID','channel','power_readings','ESSID', 'maker', 'sightings']]
print(data)

               BSSID channel power_readings        ESSID  \
0  A6:08:F5:68:D5:17       1        [-57.0]      BTWi-fi   
1  A4:08:F5:68:D3:16       1        [-57.0]  BTHub6-QTGF   
2  7A:97:41:0F:33:D7       6        [-26.0]          NaN   
3  70:97:41:0F:33:D3       6        [-27.0]    BT-8RCKHT   
4  7A:97:41:0F:33:D4       6        [-27.0]      BTWi-fi   

                    maker                                       sightings  
0                 Unknown  ['2021-06-01 18:12:35', '2021-06-01 18:12:39']  
1  Sagemcom Broadband SAS  ['2021-06-01 18:12:35', '2021-06-01 18:12:39']  
2                 Unknown  ['2021-06-01 18:12:38', '2021-06-01 18:12:39']  
3    Arcadyan Corporation  ['2021-06-01 18:12:38', '2021-06-01 18:12:39']  
4                 Unknown  ['2021-06-01 18:12:38', '2021-06-01 18:12:39']  


In [167]:
data.to_sql('test', engine, if_exists='append', index=False)

### Dezerializing listed information

Because SQL cells cannot hold lists we serialize some cell values into a string, therefore must deserialize them when we use them in the code. These properties are ones we want to track over time (ie times detected, power fluctuation over time to indicate distance from detector...)

In [168]:
def deserialize(device, detail):
    with engine.connect() as conn:
        res = conn.execute("SELECT * from test WHERE BSSID=?", (device,))
        for row in res:
            original_db_string = row[detail]
            original_db_string = original_db_string.replace("[", "")
            original_db_string = original_db_string.replace("]", "")
            original_db_string = original_db_string.replace("'", "")
            original_db_string = original_db_string.split(",")
            return original_db_string
            
    

In [169]:
with engine.connect() as conn:
    res = conn.execute("SELECT * from test")
    for row in res:
        print(type(row['power_readings']))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [170]:
dbdata_to_list = deserialize("7A:97:41:0F:33:D4", "sightings")
print(type(dbdata_to_list))
print(dbdata_to_list)

<class 'list'>
['2021-06-01 17:52:45', ' 2021-06-01 17:52:46']


**Serializing items in order to add to DB**

In [171]:
def serialize(detail):
    detail = str(detail)
    return detail

In [172]:
serialize(deserialize("7A:97:41:0F:33:D4", "sightings"))

"['2021-06-01 17:52:45', ' 2021-06-01 17:52:46']"

### Getting records already in the DB, deserializing the list data, adding the new value, reserializing and re-storing

In [173]:
def check_device(bssid):
    conn=sqlite3.connect("data_db.db")
    cur=conn.cursor()
    cur.execute("SELECT * FROM test WHERE BSSID=?", (bssid,))
    result=cur.fetchall()
    conn.close()
    
    if result != []:
        return result
    else:
        return None

dcheck = check_device("7A:97:41:0F:33:D4")

p_reading = dcheck[0][2]
s_list = dcheck[0][5]
print(type(p_reading))
print(p_reading)
print(type(s_list))
print(s_list)

<class 'str'>
[-27.0]
<class 'str'>
['2021-06-01 17:52:45', '2021-06-01 17:52:46']


### Test DataSorter class

In [174]:
from wifi_scan_datasorter import DataSorter

d = DataSorter()
d.store_new_report("../media/20210601-1817_prototype1_wifi_scan-01.csv", "test")

[HUB] DATA: Database created
simulating makers (maclookup requests exceeded)
simulating makers (maclookup requests exceeded)
simulating makers (maclookup requests exceeded)
simulating makers (maclookup requests exceeded)
simulating makers (maclookup requests exceeded)
